# op-cat

## what?

<https://github.com/bip420/bip420>

op-cat能干啥，直接上 code

        case OP_CAT:
        {
            if (stack.size() < 2)
                return set_error(serror, SCRIPT_ERR_INVALID_STACK_OPERATION);
            valtype& vch1 = stacktop(-2);
            valtype& vch2 = stacktop(-1);
            if (vch1.size() + vch2.size() > MAX_SCRIPT_ELEMENT_SIZE) // MAX_SCRIPT_ELEMENT_SIZE=520
                return set_error(serror, SCRIPT_ERR_PUSH_SIZE);
            vch1.insert(vch1.end(), vch2.begin(), vch2.end());
            stack.pop_back();
        }
        break;


功能直接了当，将stack最上面的两个元素当作字符串直接拼接起来。

## history

之前中本聪是这样实现的，基本差不多

        case OP_CAT:
        {
            // (x1 x2 -- out)
            if (stack.size() < 2)
                return false;
            valtype& vch1 = stacktop(-2);
            valtype& vch2 = stacktop(-1);
            vch1.insert(vch1.end(), vch2.begin(), vch2.end());
            stack.pop_back();
            if (stacktop(-1).size() > 5000)
                return false;
        }
        break;

不过之前的实现被删除了，为啥呢。据说是之前有个关于 **OP_LSHIFT** 的bug，会导致bitcoin节点崩溃，然后解决这个bug的时候，就把 OP_CAT 也删除了。这个bug是什么鬼？

<https://www.reddit.com/r/btc/comments/9z2shd/some_thoughts_about_op_lshiftop_rshift/>

        case OP_LSHIFT:
            if (bn2 < bnZero)
                return false;
            bn = bn1 << bn2.getulong();
            break;
            
        case OP_RSHIFT:
            if (bn2 < bnZero)
                return false;
            bn = bn1 >> bn2.getulong();
            break;

貌似是因为 **bn2** 如果很大的话，比如10个亿，会让 **bn** 超级大，然后估计就 **OOM** 了，从而产生了 **DDOS** 攻击，但因为这个就把 **OP_CAT** 干掉了？神奇。可能是中本聪被搞怕了。

具体干掉这些 op code 的commit在这里

<https://github.com/bitcoin/bitcoin/commit/4bd188c4383d6e614e18f79dc337fbabe8464c82>

        if (opcode == OP_CAT ||
            opcode == OP_SUBSTR ||
            opcode == OP_LEFT ||
            opcode == OP_RIGHT ||
            opcode == OP_INVERT ||
            opcode == OP_AND ||
            opcode == OP_OR ||
            opcode == OP_XOR ||
            opcode == OP_2MUL ||
            opcode == OP_2DIV ||
            opcode == OP_MUL ||
            opcode == OP_DIV ||
            opcode == OP_MOD ||
            opcode == OP_LSHIFT ||
            opcode == OP_RSHIFT)
            return false;


## 用处

这个 OP_CAT 搞出来有啥用呢？

### 1. an equivocation bond （翻译为 模棱两可的债券 ？？）

<https://bitcoinops.org/en/topics/op_cat/>

    The primary expected use for OP_CAT is for data provided by the creator of a script to be concatenated with data provided by someone spending from that script.

gpt翻译为：

    OP_CAT 的主要预期用途是将脚本创建者提供的数据与使用该脚本的人提供的数据进行连接。

也就是将 pay 的时候的数据，和 spend 的时候的数据拼接起来。

这里用所谓的 **equivocation bond** 来举例。 **equivocation bond** 是什么？

gpt说：

    在区块链和分布式系统中，"模棱两可债券"是一种安全机制，用来防止参与者进行不诚实的行为，特别是发出矛盾的声明或行为，例如双重支付或发表冲突的声明。这通常涉及将一定数量的加密货币或代币存入一个债券中，如果证明参与者进行了模棱两可的行为，则这些资金可能会被没收。这种机制通过损失抵押金的风险来激励参与者保持诚实。

简单来说，就是你要是作出某些类型的瞎搞行为，你会受到惩罚，比如双花。

#### 先简单说一下 **schnorr签名**

<https://en.bitcoin.it/wiki/Schnorr>

签名过程如下（删除了很多细节），可以不用看细节，核心是最后得到的签名的样子

1. 已知: 私钥 **x** ，要签名的内容 **h**
2. 先搞出公钥: **P = x * G**
3. 先一个隐藏的随机数nonce: **k**
4. 搞出公开的nonce: **R = k * G**
5. **e = hash(h || R || P)**
6. **s = k - e * x**
7. 得到签名: **(R, s)**

验证签名的算法是

1. 已知 **(R, s)**, **h**
2. **s * G = (k - e * x) * G = k * G - e * x * G = R - e * P = R - hash(h || R || P) * G**

正常情况下，我们验证签名的byte codes如下

        <签名> <pubkey> CHECK_SIG

用户在支付的时候需要提供完整的签名

有了**OP_CAT** 之后，我们可以提供这样一个脚本供别人支付

        <nonce=R> OP_CAT <pubkey> CHECK_SIG

可以看出，用户只需要提供 **s** 就可以完成支付，这样有什么好处呢？

假设用户想要双花，因为 **e** 的hash中包含了 **h**，而双花的话 **h** 肯定是不一样的，所以 **e** 也就不一样了，所以 **s** 也就不一样了。这样我们就得到了两个不同的签名 **(s1, R)** 和 **(s2, R)** , 连个签名对应同一个私钥 **x** 。而且 **s1 - s2 = (k - e1 * x) - (k - e2 * x) = (e2 - e1) * x**，所以有 **x = (s1-s2)/(e2-e1)**。

所以完蛋了， **x** 算出来了，如果这个双花的家伙还有使用 **x** 为私钥的资产的话，就暴露了。

#### more

没有 OP_CAT 能完成这个任务吗？

这个例子是不是人为性太强了？我要瞎搞的话，是不是私钥就只干这个事情，不留别的资产了？

### 2. 高级契约

